In [456]:
#importing the libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import math

In [457]:
#importing the data set

dataset = pd.read_csv("C:/Users/Mohil/Documents/Quarter 2/CS 613/spambase.data")

In [458]:
#randomizing the data set

data = dataset.iloc[:,:].values
d = len(dataset.iloc[0,:].values)
np.random.seed(0)
np.random.shuffle(data)

x = data[:,:d-1]
y = data[:,-1]

In [459]:
#binarize the feature vector using mean

mean = np.mean(x, axis=0)

for i in range(0, x.shape[1]):
    for j in range(0, x.shape[0]): 
        if(x[j][i]>=mean[i]):
            x[j][i]=1
        else:
            x[j][i]=0

In [460]:
#splitting the dataset            
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 1/3)

In [461]:
#function to calculate the overall entropy

def entropy(k, pn, p, n):

    sum = 0
    for i in range(k):
        pi = pn[i][0]
        ni = pn[i][1]
        sum += ((pi+ni)/(p+n)) * initial_entropy(pi/(pi+ni), ni/(pi+ni))
    return sum


In [462]:
#function to calculate the initial entropy

def initial_entropy(p, n):
    
    a = (-p/(p+n))*np.log2(p/(p+n))
    b = (-n/(p+n))*np.log2(n/(p+n))
    if np.isnan(a):
        a = 0
    if np.isnan(b):
        b = 0

    return a+b

In [463]:
#function to find the information gain

def gain(feature, y, p, n):

   
    unique_values = np.unique(feature)
    k = len(unique_values)
    pn = []
    for val in unique_values:
        pi = 0
        ni = 0
        for i, item in enumerate(feature):
            
            if item == val:
                if y[i] == 1:
                    pi +=1
                else:
                    ni +=1
        pn.append([pi,ni])

    return overall_entropy(p,n) - entropy(k, pn, p, n)

In [464]:
#function to find the best splitting attribute

def find_best_split(columns, y_val, p, n, done = [-1]):

    igs = []
    for i in range(columns.shape[1]):
        if i not in done:
            
            igs.append([i, gain(columns[:,i], y_val, p, n)])
    
    return sorted(igs, key = lambda x: x[1], reverse= True)[0][0]

In [465]:
#function to traverse a tree

def pretty_print_tree(root):
    stack = []
    rules = set()

    def traverse(node, stack, rules):
        if 'label' in node:
            stack.append(' THEN ' + str(node['label']))
            rules.add(''.join(stack))
            stack.pop()
        elif 'feature' in node:
            ifnd = 'IF ' if not stack else ' AND '
            stack.append(ifnd + str(node['feature']) + ' EQUALS ')
            for subnode_key in node['sub_nodes']:
                stack.append(str(subnode_key))
                traverse(node['sub_nodes'][subnode_key], stack, rules)
                stack.pop()
            stack.pop()

    traverse(root, stack, rules)


In [466]:
#function to predict the values

def predict(tree, row):

    if "label" in tree:
        y_pred.append(tree["label"])
        return y_pred
    
    elif 'feature' in tree:
        sub_nodes = tree["sub_nodes"]
        root_feature_id = tree["feature"]

        predict(tree["sub_nodes"][row[root_feature_id]], row)


In [467]:
#function to construct a decsion tree

used_features = []
tree = {}


def decision_tree(data, spam, non_spam):
   
    root = {}
    counter = Counter()
    target =  data[:, data.shape[1]-1]
    for y_val in target:
        counter[y_val] +=1
    
    if (len(counter) == 1):
        root["label"] = counter.most_common(1)[0][0]
        return root
    
    if len(used_features) == data.shape[1]-1:
        root["label"] = counter.most_common(1)[0][0]
        return root
    
    else:
       
        no_target = data[:, :data.shape[1]-1]
        best_feature_id = find_best_split(no_target, target, spam,non_spam, used_features)
        used_features.append(best_feature_id)
       
        root["feature"] = best_feature_id
        root["sub_nodes"] = {}
        x_values = set()
        x_values.update(data[:, best_feature_id])
        
        for x_value in x_values:
            
            counter = Counter()
            
            tmp = data[data[:, best_feature_id] == x_value]
           
            if len(tmp) == 0:
                for y_val in tmp[:, tmp.shape[1] - 1]:
                    counter[y_val] += 1
                root["sub_nodes"][x_value] = {"leaf": counter.most_common(1)[0][0]}
           
            else:
                #recurrsion
                root["sub_nodes"][x_value] = decision_tree(tmp, spam, non_spam)
        return root

In [468]:
#splitting the feature values into spam and not spam classes

x_spam = []
x_notspam = []

for i in range(len(x_train)):
    
    if y_train[i] == 1:
        x_spam.append(x_train[i])
   
    else:
        x_notspam.append(x_train[i])
        
x_spam = np.array(x_spam)
x_notspam =np.array(x_notspam)

In [469]:
#constructing a tree using training data set

data_set = np.append(x_train, np.array(y_train).reshape(len(y_train),1), axis=1)
tree = decision_tree(data_set, len(x_spam), len(x_notspam))

C:\Users\Mohil\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log2
  """
C:\Users\Mohil\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


In [470]:
#calculate the statistics
y_pred = []

tp = 0
tn = 0
fp = 0
fn = 0
a = 0
values =[600, 616, 840, 1034, 1167]

for i in range(0,len(y_test)):
    
    if not i in values:
        
        sample = x_test[i,:]
        y_pred.clear()
        predict(tree, sample)

        if(y_pred[0] == 1 and y_test[i] == 1):
            tp = tp+1
        if(y_pred[0] == 1 and y_test[i] == 0):
            fp = fp +1
        if(y_pred[0] == 0 and y_test[i] == 0):
            tn = tn+1
        if(y_pred[0] == 0 and y_test[i] == 1):
            fn = fn+1
        if(y_pred[0]==y_test[i]):
            a = a+1
        
precision = tp/(tp+fp)
recall = tp/(tp+fn)
f1 = 2*precision*recall/(precision+ recall)
accuracy = a/len(y_test)

print(tree)
print(precision,recall,f1,accuracy)

{'feature': 52, 'sub_nodes': {0.0: {'feature': 6, 'sub_nodes': {0.0: {'feature': 51, 'sub_nodes': {0.0: {'feature': 25, 'sub_nodes': {0.0: {'feature': 22, 'sub_nodes': {0.0: {'feature': 15, 'sub_nodes': {0.0: {'feature': 23, 'sub_nodes': {0.0: {'feature': 26, 'sub_nodes': {0.0: {'feature': 21, 'sub_nodes': {0.0: {'feature': 27, 'sub_nodes': {0.0: {'feature': 45, 'sub_nodes': {0.0: {'feature': 18, 'sub_nodes': {0.0: {'feature': 19, 'sub_nodes': {0.0: {'feature': 33, 'sub_nodes': {0.0: {'feature': 36, 'sub_nodes': {0.0: {'feature': 41, 'sub_nodes': {0.0: {'feature': 43, 'sub_nodes': {0.0: {'feature': 0, 'sub_nodes': {0.0: {'feature': 8, 'sub_nodes': {0.0: {'feature': 10, 'sub_nodes': {0.0: {'feature': 20, 'sub_nodes': {0.0: {'feature': 44, 'sub_nodes': {0.0: {'feature': 5, 'sub_nodes': {0.0: {'feature': 28, 'sub_nodes': {0.0: {'feature': 38, 'sub_nodes': {0.0: {'feature': 11, 'sub_nodes': {0.0: {'feature': 1, 'sub_nodes': {0.0: {'feature': 49, 'sub_nodes': {0.0: {'feature': 24, 'sub_node